## JSON 파일 병합 

In [1]:
import json

# 파일 경로
file1 = "/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/JSON 파일 /supplement_rag_data_fixed.json"         # 1차 파싱 성공 214개
file2 = "/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/JSON 파일 /supplements_rag_data_final8.json"       # 8차 보완 파싱 결과

# JSON 로드
with open(file1, "r", encoding="utf-8") as f1, open(file2, "r", encoding="utf-8") as f2:
    data1 = json.load(f1)
    data2 = json.load(f2)

# 병합
merged_data = data1 + data2

# 저장
output_file = "supplement_rag_data_merged.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

print(f"📦 병합된 JSON 저장 완료: {output_file}")
print(f"✅ 총 레코드 수: {len(merged_data)}개")

📦 병합된 JSON 저장 완료: supplement_rag_data_merged.json
✅ 총 레코드 수: 306개
